# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 30 2022 12:12PM,240856,10,PRF-32454,Bio-PRF,Released
1,Jun 30 2022 12:12PM,240856,10,PRF-32457,Bio-PRF,Released
2,Jun 30 2022 12:12PM,240856,10,PRF-32460,Bio-PRF,Released
3,Jun 30 2022 12:12PM,240856,10,PRF-32466,Bio-PRF,Released
4,Jun 30 2022 12:12PM,240856,10,PRF-32465,Bio-PRF,Released
5,Jun 30 2022 12:12PM,240856,10,PRF-32472,Bio-PRF,Released
6,Jun 30 2022 12:12PM,240856,10,PRF-32479,Bio-PRF,Released
7,Jun 30 2022 12:12PM,240856,10,PRF-32483,Bio-PRF,Released
8,Jun 30 2022 12:12PM,240856,10,PRF-32488,Bio-PRF,Released
9,Jun 30 2022 12:12PM,240856,10,PRF-32491,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,240851,Released,3
37,240852,Released,1
38,240853,Released,1
39,240856,Released,12
40,240857,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240851,NaN,NaN,3.0
240852,NaN,NaN,1.0
240853,NaN,NaN,1.0
240856,NaN,NaN,12.0
240857,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240750,17.0,8.0,1.0
240766,0.0,0.0,1.0
240768,0.0,0.0,1.0
240774,0.0,1.0,0.0
240781,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240750,17,8,1
240766,0,0,1
240768,0,0,1
240774,0,1,0
240781,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240750,17,8,1
1,240766,0,0,1
2,240768,0,0,1
3,240774,0,1,0
4,240781,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240750,17,8,1
1,240766,,,1
2,240768,,,1
3,240774,,1,
4,240781,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 30 2022 12:12PM,240856,10,Bio-PRF
12,Jun 30 2022 12:07PM,240857,15,"Mizner Bioscience, LLC"
15,Jun 30 2022 12:03PM,240847,15,"Person & Covey, Inc."
44,Jun 30 2022 11:51AM,240853,10,"Snap Medical Industries, LLC"
45,Jun 30 2022 11:47AM,240851,15,"Carwin Pharmaceutical Associates, LLC"
48,Jun 30 2022 11:46AM,240852,10,"Amcyte Pharma, Inc."
49,Jun 30 2022 11:46AM,240849,15,"Virtus Pharmaceuticals, LLC"
56,Jun 30 2022 11:46AM,240848,10,Beach Products Inc
67,Jun 30 2022 11:44AM,240846,22,"NBTY Global, Inc."
68,Jun 30 2022 11:43AM,240845,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 30 2022 12:12PM,240856,10,Bio-PRF,,,12
1,Jun 30 2022 12:07PM,240857,15,"Mizner Bioscience, LLC",,,3
2,Jun 30 2022 12:03PM,240847,15,"Person & Covey, Inc.",,,29
3,Jun 30 2022 11:51AM,240853,10,"Snap Medical Industries, LLC",,,1
4,Jun 30 2022 11:47AM,240851,15,"Carwin Pharmaceutical Associates, LLC",,,3
5,Jun 30 2022 11:46AM,240852,10,"Amcyte Pharma, Inc.",,,1
6,Jun 30 2022 11:46AM,240849,15,"Virtus Pharmaceuticals, LLC",,,7
7,Jun 30 2022 11:46AM,240848,10,Beach Products Inc,,,11
8,Jun 30 2022 11:44AM,240846,22,"NBTY Global, Inc.",,,1
9,Jun 30 2022 11:43AM,240845,15,"Alliance Pharma, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 12:12PM,240856,10,Bio-PRF,12,,
1,Jun 30 2022 12:07PM,240857,15,"Mizner Bioscience, LLC",3,,
2,Jun 30 2022 12:03PM,240847,15,"Person & Covey, Inc.",29,,
3,Jun 30 2022 11:51AM,240853,10,"Snap Medical Industries, LLC",1,,
4,Jun 30 2022 11:47AM,240851,15,"Carwin Pharmaceutical Associates, LLC",3,,
5,Jun 30 2022 11:46AM,240852,10,"Amcyte Pharma, Inc.",1,,
6,Jun 30 2022 11:46AM,240849,15,"Virtus Pharmaceuticals, LLC",7,,
7,Jun 30 2022 11:46AM,240848,10,Beach Products Inc,11,,
8,Jun 30 2022 11:44AM,240846,22,"NBTY Global, Inc.",1,,
9,Jun 30 2022 11:43AM,240845,15,"Alliance Pharma, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 12:12PM,240856,10,Bio-PRF,12,,
1,Jun 30 2022 12:07PM,240857,15,"Mizner Bioscience, LLC",3,,
2,Jun 30 2022 12:03PM,240847,15,"Person & Covey, Inc.",29,,
3,Jun 30 2022 11:51AM,240853,10,"Snap Medical Industries, LLC",1,,
4,Jun 30 2022 11:47AM,240851,15,"Carwin Pharmaceutical Associates, LLC",3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 12:12PM,240856,10,Bio-PRF,12.0,NaN,NaN
1,Jun 30 2022 12:07PM,240857,15,"Mizner Bioscience, LLC",3.0,NaN,NaN
2,Jun 30 2022 12:03PM,240847,15,"Person & Covey, Inc.",29.0,NaN,NaN
3,Jun 30 2022 11:51AM,240853,10,"Snap Medical Industries, LLC",1.0,NaN,NaN
4,Jun 30 2022 11:47AM,240851,15,"Carwin Pharmaceutical Associates, LLC",3.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 12:12PM,240856,10,Bio-PRF,12.0,0.0,0.0
1,Jun 30 2022 12:07PM,240857,15,"Mizner Bioscience, LLC",3.0,0.0,0.0
2,Jun 30 2022 12:03PM,240847,15,"Person & Covey, Inc.",29.0,0.0,0.0
3,Jun 30 2022 11:51AM,240853,10,"Snap Medical Industries, LLC",1.0,0.0,0.0
4,Jun 30 2022 11:47AM,240851,15,"Carwin Pharmaceutical Associates, LLC",3.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2167512,85.0,6.0,0.0
15,1445092,56.0,6.0,0.0
16,722424,3.0,6.0,0.0
17,240768,1.0,0.0,0.0
18,240813,1.0,0.0,0.0
19,722435,24.0,25.0,9.0
20,1444840,30.0,17.0,34.0
21,1203910,3.0,2.0,0.0
22,240846,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2167512,85.0,6.0,0.0
1,15,1445092,56.0,6.0,0.0
2,16,722424,3.0,6.0,0.0
3,17,240768,1.0,0.0,0.0
4,18,240813,1.0,0.0,0.0
5,19,722435,24.0,25.0,9.0
6,20,1444840,30.0,17.0,34.0
7,21,1203910,3.0,2.0,0.0
8,22,240846,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,85.0,6.0,0.0
1,15,56.0,6.0,0.0
2,16,3.0,6.0,0.0
3,17,1.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,24.0,25.0,9.0
6,20,30.0,17.0,34.0
7,21,3.0,2.0,0.0
8,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,85.0
1,15,Released,56.0
2,16,Released,3.0
3,17,Released,1.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,17,18,19,20,21,22
Status,,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,9.0,34.0,0.0,0.0
Executing,6.0,6.0,6.0,0.0,0.0,25.0,17.0,2.0,0.0
Released,85.0,56.0,3.0,1.0,1.0,24.0,30.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,9.0,34.0,0.0,0.0
1,Executing,6.0,6.0,6.0,0.0,0.0,25.0,17.0,2.0,0.0
2,Released,85.0,56.0,3.0,1.0,1.0,24.0,30.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,9.0,34.0,0.0,0.0
1,Executing,6.0,6.0,6.0,0.0,0.0,25.0,17.0,2.0,0.0
2,Released,85.0,56.0,3.0,1.0,1.0,24.0,30.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()